In [1]:
!pip install timm torchinfo
!pip install timm torchinfo git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.7/548.7 kB 8.9 MB/s eta 0:00:00a 0:00:01
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-blivisyr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-blivisyr
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369409 sha256=f03920e856fbb5ecb5530b028577f4605ca659d112cf8fae7a278238ef130955
  Stored in directory: /tmp/pip-ephem-wheel-cache-vt79b4ma/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [2]:
import timm
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torchvision import transforms  
import torch.nn.functional as F
from sklearn.decomposition import PCA
import numpy as np
import os
from tqdm import tqdm 

In [9]:
# model = timm.create_model("swin_large_patch4_window12_384_in22k", pretrained=True, num_classes=0)    
# model.to('cuda')
# print('ee')

In [3]:
import clip
from clip.clip import _download, _MODELS

# clip_code = 'ViT-L/14@336px'
# model_path = _download(_MODELS[clip_code], os.path.expanduser("~/.cache/clip"))
# with open(model_path, 'rb') as opened_file:
#     clip_vit = torch.jit.load(opened_file, map_location="cuda:0").visual.eval()
# model = clip_vit
model = timm.create_model('convnext_xlarge_in22k', pretrained=True, num_classes=0)
model.to('cuda')

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_xlarge_22k_224.pth


ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(3, 256, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((256,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(256, 256, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=256)
          (norm): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=256, out_features=1024, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (fc2): Linear(in_features=1024, out_features=256, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(256, 256, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=256)
          (norm): LayerNorm((256,), eps=1e-06, elem

In [4]:
trfs = transforms.Compose(
    [
        transforms.Resize([336, 336]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711]),

    ]
) 
dataset = torchvision.datasets.ImageFolder('../input/imagenetmini-1000/imagenet-mini/train', transform = trfs)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

embeds = []
labels = []

In [12]:
with torch.no_grad():
    for images, labels_ in tqdm(loader):
        out = model(images.to('cuda'))
        embeds.append(out)
        labels.append(labels_)

embeds = torch.cat(embeds)
labels = torch.cat(labels)

torch.save(embeds, 'embeds.pt')
torch.save(labels, 'labels.pt')

100%|██████████| 1086/1086 [1:14:49<00:00,  4.13s/it]


In [13]:
#Compute PCA on the train embeddings matrix
pca = PCA(n_components=64)
pca.fit(embeds.cpu())
from sklearn import svm
from sklearn import datasets
import pickle


# 保存模型
with open('./convnext22k.pickle', 'wb') as f:
    pickle.dump(pca,f)


In [14]:
import os
os.chdir('/kaggle/working')
print(os.getcwd())
print(os.listdir("/kaggle/working"))
from IPython.display import FileLink
FileLink('./convnext22k.pickle')

/kaggle/working
['__notebook_source__.ipynb', '.virtual_documents', 'embeds.pt', 'labels.pt', 'convnext22k.pickle']


/kaggle/working/convnext22k.pickle

In [6]:
embeds = []
labels = []
clip_code = 'ViT-L/14@336px'
model_path = _download(_MODELS[clip_code], os.path.expanduser("~/.cache/clip"))
with open(model_path, 'rb') as opened_file:
    clip_vit = torch.jit.load(opened_file, map_location="cuda:0").visual.eval()
model = clip_vit

In [7]:
with torch.no_grad():
    for images, labels_ in tqdm(loader):
        out = model(images.to('cuda').half())
        embeds.append(out)
        labels.append(labels_)

embeds = torch.cat(embeds)
labels = torch.cat(labels)

torch.save(embeds, 'embeds.pt')
torch.save(labels, 'labels.pt')

  0%|          | 0/1086 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:1110: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/BinaryOps.cpp:601.)
  return forward_call(*input, **kwargs)
100%|██████████| 1086/1086 [42:51<00:00,  2.37s/it]


In [8]:
#Compute PCA on the train embeddings matrix
pca = PCA(n_components=64)
pca.fit(embeds.cpu())
from sklearn import svm
from sklearn import datasets
import pickle


# 保存模型
with open('./clip-vit.pickle', 'wb') as f:
    pickle.dump(pca,f)

In [9]:
import os
os.chdir('/kaggle/working')
print(os.getcwd())
print(os.listdir("/kaggle/working"))
from IPython.display import FileLink
FileLink('./clip-vit.pickle')

/kaggle/working
['embeds.pt', '.virtual_documents', '__notebook_source__.ipynb', 'clip-vit.pickle', 'labels.pt']


/kaggle/working/clip-vit.pickle

In [ ]:
embeds.shape[1]

In [ ]:
# linear test 
#l = nn.Linear(embeds.shape[1], 64, bias=False)
#l.weight = torch.nn.Parameter(torch.tensor(pca_comp))

In [ ]:
class MyModel(nn.Module):
  def __init__(self, model_name, target_size=[224, 224], emb_dim=1536, pca_weights=None, normalize=True):
    super().__init__()
    self.target_size = target_size
    
    self.encoder = timm.create_model(model_name, pretrained=True, num_classes=0)    
    
    if pca_weights is None: 
        self.final = nn.AdaptiveAvgPool1d(64)
    else: 
        self.final =  nn.Linear(emb_dim, 64, bias=False)
        self.final.weight = torch.nn.Parameter(torch.tensor(pca_weights))
        
    self.normalize = normalize

  def forward(self, x):
    x = transforms.functional.resize(x,size=self.target_size)
    x = x / 255.0
    x = transforms.functional.normalize(x, 
                                            mean=[0.485, 0.456, 0.406], 
                                            std=[0.229, 0.224, 0.225])
    x = self.encoder(x)
    x = self.final(x)
    
    if self.normalize:
        x = torch.nn.functional.normalize(x)
    return x

In [ ]:
m = MyModel("swin_large_patch4_window12_384_in22k", target_size=[384, 384], emb_dim=embeds.shape[1], pca_weights=pca_comp, normalize=True)
m.eval();

In [ ]:
saved_model = torch.jit.script(m)
saved_model.save('saved_model.pt')

In [ ]:
check_model = torch.jit.load('saved_model.pt')

print(check_model)

In [ ]:
!ls -lh

In [ ]:
from zipfile import ZipFile

with ZipFile('submission.zip','w') as zip:           
  zip.write("./saved_model.pt", arcname='saved_model.pt') 

In [ ]:
!ls -lh

In [ ]:
from torchinfo import summary

test_input_size = (2, 3, 384, 384)  # the model should work with any input_size
summary(model, input_size=test_input_size)

In [ ]:
saved_model = torch.jit.load('saved_model.pt').cuda()
input = torch.ones(test_input_size, device='cuda')

assert saved_model(input).shape == torch.Size([2, 64])